# Обучение модели Real-ESRGAN для восстановления изображений с эффектом "пикселизации"
Этот Notebook демонстрирует процесс fine-tuning модели Real-ESRGAN на датасете с искаженными изображениями (эффект пикселизации). 

Мы используем предобученные веса с scale=1 (без upscale, только denoising/depixelation) и адаптируем модель под разрешение 1024x1448. 

Обучение включает комбинированный loss (Charbonnier + perceptual), мониторинг метрик (Loss, PSNR, SSIM), early stopping и сохранение чекпоинтов.

## Шаг 1: Настройка окружения и выбор GPU
Выбираем свободную GPU, настраиваем PyTorch для оптимальной работы с CUDA.

In [ ]:
import os
import subprocess
import torch
from torch.amp import autocast, GradScaler
import time
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Выбор GPU
result = subprocess.check_output(["nvidia-smi", "--query-gpu=index,memory.free", "--format=csv,noheader,nounits"]).decode().strip()
gpu_list = [tuple(map(int, line.split(', '))) for line in result.splitlines()]
best_idx, max_free = max(gpu_list, key=lambda x: x[1])
os.environ['CUDA_VISIBLE_DEVICES'] = str(best_idx)
print(f"Выбрана GPU {best_idx} — свободно ≈{max_free//1024} GiB")

device = torch.device('cuda:0')
torch.cuda.set_device(device)
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.benchmark = True
torch.cuda.empty_cache()

## Шаг 2: Импорт архитектуры модели
Переходим в директорию Real-ESRGAN, импортируем RRDBNet с попытками для стабильности.

In [ ]:
os.chdir('/home/kudriavtcevroman-10/Real-ESRGAN')
max_attempts = 5
attempt = 0
success = False
while attempt < max_attempts and not success:
    try:
        from basicsr.archs.rrdbnet_arch import RRDBNet
        success = True
        print("Импорт удался на попытке", attempt + 1)
    except ImportError as e:
        attempt += 1
        print(f"Ошибка на попытке {attempt}: {e}. Повтор через 2 сек...")
        time.sleep(2)

if not success:
    raise ImportError("Импорт не удался после max_attempts попыток. Проверьте установку basicsr или зависимости.")
os.chdir('/home/kudriavtcevroman-10')

## Шаг 3: Инициализация модели и загрузка весов
Инициализируем RRDBNet с параметрами для depixelation (scale=1), загружаем предобученные веса частично.

In [ ]:
model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=20, num_grow_ch=32, scale=1).to(device)

# Загрузка весов
weights_path = '/home/kudriavtcevroman-10/pretrained/RealESRGAN_x4plus.pth'
checkpoint = torch.load(weights_path, map_location=device)
pretrained_state = checkpoint['params_ema']

model_state = model.state_dict()
for k, v in pretrained_state.items():
    if k in model_state and v.size() == model_state[k].size():
        model_state[k] = v

model.load_state_dict(model_state, strict=False)
print(f"Модель на устройстве: {device}. Веса загружены частично с фильтрацией (strict=False). Новые слои дообучатся.")
print(f"VRAM после модели: {torch.cuda.memory_allocated(device)/1024**3:.1f} GiB")

## Шаг 4: Подготовка датасета
Определяем кастомный датасет для distorted/clean изображений, с трансформацией на разрешение 1024x1448 и padding.

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

# Кастомный трансформер
class PaddedResize:
    def __init__(self, size=(1024, 1448), div_factor=8):
        self.size = size
        self.div_factor = div_factor

    def __call__(self, img):
        img = transforms.Resize(self.size, interpolation=Image.LANCZOS)(img)
        w, h = img.size
        pad_w = (self.div_factor - w % self.div_factor) % self.div_factor
        pad_h = (self.div_factor - h % self.div_factor) % self.div_factor
        if pad_w != 0 or pad_h != 0:
            img = transforms.Pad((0, 0, pad_w, pad_h), fill=0)(img)
        return img

# Трансформации
transform = transforms.Compose([
    PaddedResize(size=(1024, 1448)),
    transforms.ToTensor(),
])

# Кастомный датасет
class PixelationDataset(Dataset):
    def __init__(self, distorted_dir, clean_dir, transform=None):
        self.distorted_images = sorted(os.listdir(distorted_dir))
        self.clean_images = sorted(os.listdir(clean_dir))
        self.distorted_dir = distorted_dir
        self.clean_dir = clean_dir
        self.transform = transform

    def __len__(self): return len(self.distorted_images)

    def __getitem__(self, idx):
        dist = Image.open(os.path.join(self.distorted_dir, self.distorted_images[idx])).convert('RGB')
        clean = Image.open(os.path.join(self.clean_dir, self.clean_images[idx])).convert('RGB')
        if self.transform:
            dist = self.transform(dist)
            clean = self.transform(clean)
        return dist, clean

# Датасеты и лоадеры
train_dataset = PixelationDataset('pixelation/train/distorted', 'pixelation/train/clean', transform=transform)
val_dataset = PixelationDataset('pixelation/val/distorted', 'pixelation/val/clean', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=4, pin_memory=True)

print(f"Train: {len(train_dataset)}, Val: {len(val_dataset)}")

## Шаг 5: Настройка loss и обучения
Определяем combined loss (Charbonnier + perceptual), метрики, оптимизатор, scheduler и early stopping.

In [ ]:
import torch.nn as nn
from torchvision.models import vgg19
from torchmetrics.image import PeakSignalNoiseRatio, StructuralSimilarityIndexMeasure
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from tqdm import tqdm

# Perceptual loss
vgg = vgg19(pretrained=True).features.to(device).eval()
for param in vgg.parameters():
    param.requires_grad = False

def perceptual_loss(output, clean):
    def get_features(x):
        return vgg[:23](x)  # conv4_4
    return nn.L1Loss()(get_features(output), get_features(clean))

# Charbonnier loss
def charbonnier_loss(pred, target, eps=1e-6):
    return torch.mean(torch.sqrt((pred - target) ** 2 + eps))

# Combined loss
def combined_loss(output, clean):
    return 0.7 * charbonnier_loss(output, clean) + 0.3 * perceptual_loss(output, clean)

# Метрики
psnr_metric = PeakSignalNoiseRatio(data_range=1.0).to(device)
ssim_metric = StructuralSimilarityIndexMeasure(data_range=1.0).to(device)

# Оптимизатор и scaler
optimizer = AdamW(model.parameters(), lr=2e-4, weight_decay=1e-4)
scaler = GradScaler()

# Scheduler
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2, eta_min=1e-6)

# Early stopping
patience = 10
early_stop_counter = 0
best_val_psnr = -float('inf')

# Директория для чекпоинтов
os.makedirs('checkpoints', exist_ok=True)
checkpoint_path = 'checkpoints/real_esrgan_pix_ep{}.pth'

# Списки для метрик
train_losses, val_losses = [], []
train_psnrs, val_psnrs = [], []
train_ssims, val_ssims = [], []

num_epochs = 300

## Шаг 6: Цикл обучения
Обучаем модель с mixed precision, мониторим метрики, сохраняем чекпоинты и применяем early stopping.

In [ ]:
for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    train_loss = train_psnr = train_ssim = 0.0

    for dist, clean in tqdm(train_loader, desc=f"Epoch {epoch+1:02d}/{num_epochs} [train]"):
        dist = dist.to(device, non_blocking=True)
        clean = clean.to(device, non_blocking=True)
        optimizer.zero_grad()
        with autocast(device_type='cuda'):
            pred = model(dist)
            loss = combined_loss(pred, clean)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        train_loss += loss.item()
        with torch.no_grad():
            train_psnr += psnr_metric(pred, clean).item()
            train_ssim += ssim_metric(pred, clean).item()

    avg_train_loss = train_loss / len(train_loader)
    avg_train_psnr = train_psnr / len(train_loader)
    avg_train_ssim = train_ssim / len(train_loader)
    train_losses.append(avg_train_loss)
    train_psnrs.append(avg_train_psnr)
    train_ssims.append(avg_train_ssim)

    scheduler.step()

    # Валидация
    model.eval()
    val_loss = val_psnr = val_ssim = 0.0
    with torch.no_grad():
        for dist, clean in tqdm(val_loader, desc=f"Epoch {epoch+1:02d}/{num_epochs} [val]"):
            dist = dist.to(device, non_blocking=True)
            clean = clean.to(device, non_blocking=True)
            with autocast(device_type='cuda'):
                pred = model(dist)
                val_loss += combined_loss(pred, clean).item()
                val_psnr += psnr_metric(pred, clean).item()
                val_ssim += ssim_metric(pred, clean).item()

    avg_val_loss = val_loss / len(val_loader)
    avg_val_psnr = val_psnr / len(val_loader)
    avg_val_ssim = val_ssim / len(val_loader)
    val_losses.append(avg_val_loss)
    val_psnrs.append(avg_val_psnr)
    val_ssims.append(avg_val_ssim)

    print(f"\nЭпоха {epoch+1:02d} | Train Loss: {avg_train_loss:.5f} | Val Loss: {avg_val_loss:.5f} | "
          f"Val PSNR: {avg_val_psnr:.2f} dB | Val SSIM: {avg_val_ssim:.4f} | Время: {time.time() - start_time:.2f} с | "
          f"VRAM: {torch.cuda.memory_allocated(device)/1024**3:.1f} GiB")

    # Сохранение чекпоинта
    torch.save({
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'best_val_psnr': best_val_psnr,
    }, checkpoint_path.format(epoch+1))
    print(f"Чекпоинт сохранён для эпохи {epoch+1}")

    # Early stopping
    if avg_val_psnr > best_val_psnr:
        best_val_psnr = avg_val_psnr
        torch.save(model.state_dict(), "checkpoints/real_esrgan_best_psnr.pth")
        print("    → Лучшая модель сохранена!")
        early_stop_counter = 0
    else:
        early_stop_counter += 1
        print(f"    → Нет улучшения ({early_stop_counter}/{patience})")

    if early_stop_counter >= patience:
        print(f"Early stopping после {patience} эпох без улучшения.")
        break

    torch.cuda.empty_cache()

print("Fine-tuning Real-ESRGAN завершён")

## Шаг 7: Сохранение финальной модели
Сохраняем обученные веса для дальнейшего использования в инференсе.

In [ ]:
torch.save(model.state_dict(), 'finetuned_real_esrgan.pth')
print("Финальная модель сохранена как finetuned_real_esrgan.pth.")

## Шаг 8: Визуализация метрик
Строим графики для анализа сходимости и проверки на переобучение.

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(3, 1, figsize=(10, 15))

# Loss
axs[0].plot(range(1, len(train_losses)+1), train_losses, label='Train Loss', marker='o')
axs[0].plot(range(1, len(val_losses)+1), val_losses, label='Val Loss', marker='o')
axs[0].set_title('Loss over Epochs')
axs[0].set_xlabel('Epoch')
axs[0].set_ylabel('Loss')
axs[0].legend()
axs[0].grid(True)

# PSNR
axs[1].plot(range(1, len(train_psnrs)+1), train_psnrs, label='Train PSNR', marker='o')
axs[1].plot(range(1, len(val_psnrs)+1), val_psnrs, label='Val PSNR', marker='o')
axs[1].set_title('PSNR over Epochs')
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('PSNR (dB)')
axs[1].legend()
axs[1].grid(True)

# SSIM
axs[2].plot(range(1, len(train_ssims)+1), train_ssims, label='Train SSIM', marker='o')
axs[2].plot(range(1, len(val_ssims)+1), val_ssims, label='Val SSIM', marker='o')
axs[2].set_title('SSIM over Epochs')
axs[2].set_xlabel('Epoch')
axs[2].set_ylabel('SSIM')
axs[2].legend()
axs[2].grid(True)

plt.tight_layout()
plt.savefig('real_esrgan_metrics.png')
plt.show()
print("Графики сохранены как real_esrgan_metrics.png. Проверьте val на рост (PSNR >30-35 dB, SSIM >0.9 идеально для пикселизации).")